In [ ]:
import ot  # Python Optimal Transport library
from scipy.optimize import minimize
from scipy.stats import multivariate_normal

def fit_wasserstein_prior(posterior_draws, prior_family='mvn'):
    """
    Fit a parametric prior by minimizing Wasserstein distance
    to empirical posterior draws
    """
    D = posterior_draws.shape[1]
    
    if prior_family == 'mvn':
        # Optimize over multivariate normal parameters
        def objective(params):
            mu = params[:D]
            # Ensure positive definite covariance (Cholesky parameterization)
            L_vec = params[D:]
            L = np.zeros((D, D))
            L[np.triu_indices(D)] = L_vec
            Sigma = L @ L.T + 1e-6 * np.eye(D)  # Add small regularization
            
            # Sample from fitted distribution
            fitted_samples = multivariate_normal.rvs(mu, Sigma, size=len(posterior_draws))
            
            # Compute Wasserstein-2 distance
            cost_matrix = ot.dist(posterior_draws, fitted_samples, metric='euclidean')
            W2_distance = ot.emd2([], [], cost_matrix)
            return W2_distance
        
        # Initialize with sample mean and covariance
        init_mu = np.mean(posterior_draws, axis=0)
        init_cov = np.cov(posterior_draws.T)
        L_init = np.linalg.cholesky(init_cov)
        init_params = np.concatenate([init_mu, L_init[np.triu_indices(D)]])
        
        # Optimize
        result = minimize(objective, init_params, method='L-BFGS-B')
        
        # Extract optimized parameters
        mu_opt = result.x[:D]
        L_vec_opt = result.x[D:]
        L_opt = np.zeros((D, D))
        L_opt[np.triu_indices(D)] = L_vec_opt
        Sigma_opt = L_opt @ L_opt.T
        
        return mu_opt, Sigma_opt

# After fitting OH model:
beta_OH_draws = fit_OH.stan_variable('beta')

# Fit Wasserstein-optimal prior
mu_prior, Sigma_prior = fit_wasserstein_prior(beta_OH_draws)

# Use in Stan model (modified version)
data_NY_wass = {
    'N': N, 'D': D, 'x': x_NY, 'y': y_NY,
    'mu_prior': mu_prior, 'Sigma_prior': Sigma_prior
}

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression

def simulate_once(n=1000, random_state=None):
    rng = np.random.RandomState(random_state)
    
    # Simulate covariate X, instrument Z, and treatment D
    X = rng.uniform(-1, 1, size=(n, 1))
    Z = rng.binomial(1, 0.5, size=(n, 1))
    logits = 1.0 * Z.ravel() + 0.5 * X.ravel()
    p_true = 1 / (1 + np.exp(-logits))
    D = rng.binomial(1, p_true)
    
    # Outcome Y with true beta = 2
    beta_true = 2.0
    Y = beta_true * D + rng.normal(0, 1, size=n)
    
    # First-stage ML: Random Forest classifier for D ~ (Z, X)
    features = np.hstack([Z, X])
    clf = RandomForestClassifier(n_estimators=50, random_state=rng)
    clf.fit(features, D)
    p_hat = clf.predict_proba(features)[:, 1]
    
    # Isotonic calibration
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(p_hat, D)
    p_cal = iso.predict(p_hat)
    
    # Second-stage estimation: regress Y on fitted values
    lr_raw = LinearRegression().fit(p_hat.reshape(-1, 1), Y)
    lr_cal = LinearRegression().fit(p_cal.reshape(-1, 1), Y)
    
    return lr_raw.coef_[0], lr_cal.coef_[0]

# Run simulations
n_sim = 100
results = np.array([simulate_once(random_state=i) for i in range(n_sim)])
errors = (results - 2.0) ** 2

# Compute MSE for raw vs calibrated first stage
mse_raw = errors[:, 0].mean()
mse_cal = errors[:, 1].mean()

df = pd.DataFrame({
    'Method': ['Raw ML First Stage', 'Isotonic-Calibrated'],
    'MSE of β̂': [mse_raw, mse_cal]
})


In [2]:
df

,Method,MSE of β̂
0,Raw ML First Stage,0.363006
1,Isotonic-Calibrated,0.004333


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.isotonic import IsotonicRegression

def simulate_iv(n=1000, random_state=None):
    rng = np.random.RandomState(random_state)
    
    # Simulate covariate X, instrument Z, and treatment D
    X = rng.uniform(-1, 1, size=(n, 1))
    Z = rng.binomial(1, 0.5, size=(n, 1))
    logits = 1.0 * Z.ravel() + 0.5 * X.ravel()
    p_true = 1 / (1 + np.exp(-logits))
    D = rng.binomial(1, p_true)
    
    # Outcome Y with true beta = 2
    beta_true = 2.0
    Y = beta_true * D + rng.normal(0, 1, size=n)
    
    # First-stage ML: Random Forest classifier for D ~ (Z, X)
    features = np.hstack([Z, X])
    clf = RandomForestClassifier(n_estimators=50, random_state=rng)
    clf.fit(features, D)
    p_hat = clf.predict_proba(features)[:, 1]
    
    # Isotonic calibration
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(p_hat, D)
    p_cal = iso.predict(p_hat)
    
    # Second-stage OLS using predicted probabilities as regressors
    def second_stage(regressor):
        X2 = sm.add_constant(regressor)
        model = sm.OLS(Y, X2).fit()
        coef = model.params[1]
        se = model.bse[1]
        return coef, se
    
    coef_raw, se_raw = second_stage(p_hat)
    coef_cal, se_cal = second_stage(p_cal)
    
    return coef_raw, se_raw, coef_cal, se_cal

# Run simulations
n_sim = 100
results = np.array([simulate_iv(random_state=i) for i in range(n_sim)])
coefs = results[:, [0, 2]]
ses = results[:, [1, 3]]
errors = (coefs - 2.0) ** 2

mse_raw = errors[:, 0].mean()
mse_cal = errors[:, 1].mean()
se_mean_raw = ses[:, 0].mean()
se_mean_cal = ses[:, 1].mean()

df = pd.DataFrame({
    'Method': ['Raw ML First Stage', 'Isotonic-Calibrated'],
    'MSE of β̂': [mse_raw, mse_cal],
    'Avg. SE of β̂': [se_mean_raw, se_mean_cal]
})

df

,Method,MSE of β̂,Avg. SE of β̂
0,Raw ML First Stage,0.363006,0.097222
1,Isotonic-Calibrated,0.004333,0.065673


In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

def simulate_2sls_data(n=1000, z_coef=0.5, x_coef=0.5, beta_true=2.0, random_state=None):
    """Simulate data for 2SLS with binary treatment"""
    rng = np.random.RandomState(random_state)
    
    # Covariates and instruments
    X = rng.uniform(-1, 1, size=n)
    Z = rng.binomial(1, 0.5, size=n)
    
    # Treatment equation: D = f(Z, X, u)
    logits = z_coef * Z + x_coef * X
    prob_D = 1 / (1 + np.exp(-logits))
    D = rng.binomial(1, prob_D)
    
    # Outcome equation: Y = beta*D + X + error
    # Include X in outcome to create endogeneity if it affects both D and Y
    error = rng.normal(0, 1, size=n)
    Y = beta_true * D + 1.0 * X + error
    
    return {'X': X, 'Z': Z, 'D': D, 'Y': Y, 'prob_D_true': prob_D}

def run_proper_2sls(Y, D, Z, X, d_hat):
    """Run proper 2SLS using fitted values from first stage"""
    n = len(Y)
    
    # Second stage: regress Y on D_hat and X
    X_matrix = np.column_stack([np.ones(n), d_hat, X])
    
    # Compute 2SLS estimator
    try:
        beta_hat = np.linalg.solve(X_matrix.T @ X_matrix, X_matrix.T @ Y)
        
        # Compute standard errors (simplified - doesn't account for first-stage uncertainty)
        residuals = Y - X_matrix @ beta_hat
        sigma2 = np.sum(residuals**2) / (n - X_matrix.shape[1])
        var_beta = sigma2 * np.linalg.inv(X_matrix.T @ X_matrix)
        se_beta = np.sqrt(np.diag(var_beta))
        
        return beta_hat[1], se_beta[1]  # Return coefficient on D_hat
    except:
        return np.nan, np.nan

def compute_first_stage_stats(D, Z, X, d_hat):
    """Compute first-stage F-statistic and partial R-squared"""
    n = len(D)
    
    # Regression of D on Z, X
    X_full = np.column_stack([np.ones(n), Z, X])
    
    try:
        # OLS coefficients
        coefs = np.linalg.solve(X_full.T @ X_full, X_full.T @ D)
        
        # Fitted values and residuals
        D_fitted = X_full @ coefs
        residuals_full = D - D_fitted
        
        # Regression of D on just X (restricted model)
        X_restricted = np.column_stack([np.ones(n), X])
        coefs_restricted = np.linalg.solve(X_restricted.T @ X_restricted, X_restricted.T @ D)
        D_fitted_restricted = X_restricted @ coefs_restricted
        residuals_restricted = D - D_fitted_restricted
        
        # F-statistic for testing Z's significance
        rss_full = np.sum(residuals_full**2)
        rss_restricted = np.sum(residuals_restricted**2)
        f_stat = ((rss_restricted - rss_full) / 1) / (rss_full / (n - 3))
        
        # Partial R-squared of Z
        partial_r2 = (rss_restricted - rss_full) / rss_restricted
        
        # R-squared using d_hat predictions
        r2_dhat = 1 - np.sum((D - d_hat)**2) / np.sum((D - np.mean(D))**2)
        
        return f_stat, partial_r2, r2_dhat
    except:
        return np.nan, np.nan, np.nan

def calibration_diagnostics(y_true, y_prob):
    """Compute calibration diagnostics"""
    # Bin predictions and compute empirical frequencies
    n_bins = 10
    bins = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    digitized = np.digitize(y_prob, bins) - 1
    digitized = np.clip(digitized, 0, n_bins - 1)
    
    empirical_prob = np.zeros(n_bins)
    pred_prob = np.zeros(n_bins)
    
    for i in range(n_bins):
        mask = digitized == i
        if np.sum(mask) > 0:
            empirical_prob[i] = np.mean(y_true[mask])
            pred_prob[i] = np.mean(y_prob[mask])
    
    # Brier score and calibration error
    brier_score = np.mean((y_prob - y_true)**2)
    calibration_error = np.mean(np.abs(pred_prob - empirical_prob))
    
    return {
        'brier_score': brier_score,
        'calibration_error': calibration_error,
        'bin_centers': bin_centers,
        'empirical_prob': empirical_prob,
        'pred_prob': pred_prob
    }

def simulation_study():
    """Run comprehensive simulation study"""
    n_sim = 200
    n_obs = 1000
    scenarios = [
        {'z_coef': 0.5, 'name': 'Strong instrument'},
        {'z_coef': 0.2, 'name': 'Weak instrument'},
        {'z_coef': 1.0, 'name': 'Very strong instrument'}
    ]
    
    results = []
    
    for scenario in scenarios:
        print(f"Running scenario: {scenario['name']}")
        
        for sim in range(n_sim):
            # Generate data
            data = simulate_2sls_data(
                n=n_obs, 
                z_coef=scenario['z_coef'],
                random_state=sim
            )
            
            X, Z, D, Y = data['X'], data['Z'], data['D'], data['Y']
            
            # Stack features for ML
            features = np.column_stack([Z, X])
            
            # Method 1: Linear 2SLS (baseline)
            X_fs = np.column_stack([np.ones(len(D)), Z, X])
            coefs_linear = np.linalg.solve(X_fs.T @ X_fs, X_fs.T @ D)
            d_hat_linear = X_fs @ coefs_linear
            beta_linear, se_linear = run_proper_2sls(Y, D, Z, X, d_hat_linear)
            f_stat_linear, partial_r2_linear, r2_linear = compute_first_stage_stats(D, Z, X, d_hat_linear)
            
            # Method 2: Random Forest (raw)
            rf = RandomForestClassifier(n_estimators=100, random_state=sim, max_depth=5)
            rf.fit(features, D)
            p_hat_rf = rf.predict_proba(features)[:, 1]
            beta_rf_raw, se_rf_raw = run_proper_2sls(Y, D, Z, X, p_hat_rf)
            f_stat_rf_raw, partial_r2_rf_raw, r2_rf_raw = compute_first_stage_stats(D, Z, X, p_hat_rf)
            
            # Calibration diagnostics for raw RF
            cal_diag_raw = calibration_diagnostics(D, p_hat_rf)
            
            # Method 3: Random Forest + Isotonic calibration
            iso = IsotonicRegression(out_of_bounds='clip')
            iso.fit(p_hat_rf, D)
            p_hat_rf_cal = iso.predict(p_hat_rf)
            beta_rf_cal, se_rf_cal = run_proper_2sls(Y, D, Z, X, p_hat_rf_cal)
            f_stat_rf_cal, partial_r2_rf_cal, r2_rf_cal = compute_first_stage_stats(D, Z, X, p_hat_rf_cal)
            
            # Calibration diagnostics for calibrated RF
            cal_diag_cal = calibration_diagnostics(D, p_hat_rf_cal)
            
            # Method 4: Logistic regression (should be well-calibrated)
            lr = LogisticRegression(random_state=sim)
            lr.fit(features, D)
            p_hat_lr = lr.predict_proba(features)[:, 1]
            beta_lr, se_lr = run_proper_2sls(Y, D, Z, X, p_hat_lr)
            f_stat_lr, partial_r2_lr, r2_lr = compute_first_stage_stats(D, Z, X, p_hat_lr)
            
            # Store results
            for method, beta, se, f_stat, partial_r2, r2, cal_error in [
                ('Linear 2SLS', beta_linear, se_linear, f_stat_linear, partial_r2_linear, r2_linear, 0),
                ('RF Raw', beta_rf_raw, se_rf_raw, f_stat_rf_raw, partial_r2_rf_raw, r2_rf_raw, cal_diag_raw['calibration_error']),
                ('RF Calibrated', beta_rf_cal, se_rf_cal, f_stat_rf_cal, partial_r2_rf_cal, r2_rf_cal, cal_diag_cal['calibration_error']),
                ('Logistic', beta_lr, se_lr, f_stat_lr, partial_r2_lr, r2_lr, 0)
            ]:
                results.append({
                    'scenario': scenario['name'],
                    'z_coef': scenario['z_coef'],
                    'sim': sim,
                    'method': method,
                    'beta_hat': beta,
                    'se': se,
                    'f_stat': f_stat,
                    'partial_r2': partial_r2,
                    'r2': r2,
                    'calibration_error': cal_error,
                    'bias': beta - 2.0 if not np.isnan(beta) else np.nan,
                    'mse': (beta - 2.0)**2 if not np.isnan(beta) else np.nan
                })
    
    return pd.DataFrame(results)

def analyze_results(results_df):
    """Analyze and display simulation results"""
    
    # Summary statistics by method and scenario
    summary = results_df.groupby(['scenario', 'method']).agg({
        'beta_hat': ['mean', 'std'],
        'bias': 'mean',
        'mse': 'mean',
        'f_stat': 'mean',
        'partial_r2': 'mean',
        'r2': 'mean',
        'calibration_error': 'mean'
    }).round(4)
    
    summary.columns = ['Beta Mean', 'Beta Std', 'Bias', 'MSE', 'F-stat', 'Partial R²', 'R²', 'Cal Error']
    
    print("=== Simulation Results Summary ===")
    print(summary)
    
    # Focus on strong instrument scenario for main results
    strong_results = results_df[results_df['scenario'] == 'Strong instrument']
    
    main_summary = strong_results.groupby('method').agg({
        'f_stat': 'mean',
        'partial_r2': 'mean', 
        'mse': 'mean',
        'se': 'mean'
    }).round(3)
    
    print("\n=== Main Results (Strong Instrument) ===")
    print(main_summary[['f_stat', 'partial_r2', 'mse']])
    
    return summary

# Run the simulation
if __name__ == "__main__":
    print("Starting simulation study...")
    results = simulation_study()
    
    print("\nAnalyzing results...")
    summary = analyze_results(results)
    
    # Save results
    results.to_csv('2sls_calibration_results.csv', index=False)

Starting simulation study...
Running scenario: Strong instrument
Running scenario: Weak instrument
Running scenario: Very strong instrument

Analyzing results...
=== Simulation Results Summary ===
                                      Beta Mean  Beta Std    Bias        MSE  \
scenario               method                                                  
Strong instrument      Linear 2SLS       2.0406    0.5746  0.0406     0.3302   
                       Logistic          2.0792    0.5827  0.0792     0.3441   
                       RF Calibrated     2.3561    0.2487  0.3561     0.1884   
                       RF Raw            4.1458    0.5849  2.1458     4.9449   
Very strong instrument Linear 2SLS       2.0229    0.2823  0.0229     0.0798   
                       Logistic          2.0616    0.2854  0.0616     0.0848   
                       RF Calibrated     2.1499    0.1745  0.1499     0.0528   
                       RF Raw            3.1185    0.3294  1.1185     1.3589   
Wea

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

def simulate_2sls_data(n=1000, z_coef=0.5, x_coef=0.5, beta_true=2.0, random_state=None):
    rng = np.random.RandomState(random_state)
    X = rng.uniform(-1, 1, size=n)
    Z = rng.binomial(1, 0.5, size=n)
    logits = z_coef * Z + x_coef * X
    p_D = 1 / (1 + np.exp(-logits))
    D = rng.binomial(1, p_D)
    Y = beta_true * D + 1.0 * X + rng.normal(0, 1, size=n)
    return X, Z, D, Y

def first_stage_ml(Z, X, D, calibrate=False):
    features = np.column_stack([Z, X])
    rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
    rf.fit(features, D)
    p_hat = rf.predict_proba(features)[:,1]
    if calibrate:
        iso = IsotonicRegression(out_of_bounds='clip')
        iso.fit(p_hat, D)
        p_hat = iso.predict(p_hat)
    return p_hat

def run_2sls(Y, D_hat, X):
    # Equivalent to 2SLS with instrument Z: second stage regresses on fitted values
    n = len(Y)
    design = np.column_stack([np.ones(n), D_hat, X])
    coef = np.linalg.lstsq(design, Y, rcond=None)[0]
    resid = Y - design.dot(coef)
    sigma2 = (resid**2).sum() / (n - design.shape[1])
    cov = sigma2 * np.linalg.inv(design.T.dot(design))
    return coef[1], np.sqrt(cov[1,1])

def first_stage_stats(D, Z, X, D_hat):
    n = len(D)
    # Full model
    X_full = sm.add_constant(np.column_stack([Z, X]))
    res_full = sm.OLS(D, X_full).fit()
    # Restricted (no Z)
    X_res = sm.add_constant(X)
    res_res = sm.OLS(D, X_res).fit()
    f_stat = ((res_res.ssr - res_full.ssr) / 1) / (res_full.ssr / (n - X_full.shape[1]))
    part_r2 = (res_res.ssr - res_full.ssr) / res_res.ssr
    # R2 of D_hat
    r2_hat = 1 - np.sum((D - D_hat)**2) / np.sum((D - D.mean())**2)
    return f_stat, part_r2, r2_hat

def simulation_study(n_sim=200, n=1000):
    scenarios = [
        (0.5, 'Strong instrument'),
        (0.2, 'Weak instrument'),
        (1.0, 'Very strong instrument')
    ]
    records = []
    for z_coef, name in scenarios:
        for sim in range(n_sim):
            X, Z, D, Y = simulate_2sls_data(n=n, z_coef=z_coef, random_state=sim)
            # Raw ML
            D_hat_raw = first_stage_ml(Z, X, D, calibrate=False)
            # Calibrated ML
            D_hat_cal = first_stage_ml(Z, X, D, calibrate=True)
            # Linear 2SLS baseline
            D_hat_lin, _ = run_2sls(D, sm.add_constant(np.column_stack([Z, X])).dot(np.linalg.lstsq(sm.add_constant(np.column_stack([Z, X])), D, rcond=None)[0]), X)
            # Actually, baseline uses OLS fitted values
            lin_fs = sm.OLS(D, sm.add_constant(np.column_stack([Z, X]))).fit()
            D_hat_lin = lin_fs.fittedvalues
            
            for method, D_hat in [
                ('Linear 2SLS', D_hat_lin),
                ('RF Raw', D_hat_raw),
                ('RF Calibrated', D_hat_cal)
            ]:
                beta, se = run_2sls(Y, D_hat, X)
                f_stat, p_r2, r2 = first_stage_stats(D, Z, X, D_hat)
                records.append({
                    'scenario': name,
                    'method': method,
                    'beta_hat': beta,
                    'se_beta': se,
                    'f_stat': f_stat,
                    'partial_r2': p_r2,
                    'r2_hat': r2,
                    'bias': beta - 2.0,
                    'mse': (beta - 2.0)**2
                })
    return pd.DataFrame(records)

# Run and summarize
df = simulation_study()
summary = df.groupby(['scenario','method']).agg({
    'beta_hat':['mean','std'],
    'bias':'mean',
    'mse':'mean',
    'se_beta':'mean',
    'f_stat':'mean',
    'partial_r2':'mean'
}).round(3)
summary

beta_hat           bias       mse  \
                                         mean     std   mean      mean   
scenario               method                                            
Strong instrument      Linear 2SLS      2.041   0.575  0.041     0.330   
                       RF Calibrated    2.360   0.249  0.360     0.191   
                       RF Raw           4.148   0.583  2.148     4.951   
Very strong instrument Linear 2SLS      2.023   0.282  0.023     0.080   
                       RF Calibrated    2.149   0.174  0.149     0.052   
                       RF Raw           3.122   0.326  1.122     1.366   
Weak instrument        Linear 2SLS      0.618  33.532 -1.382  1120.672   
                       RF Calibrated    2.543   0.316  0.543     0.394   
                       RF Raw           4.961   0.699  2.961     9.255   

                                     se_beta  f_stat partial_r2  
                                        mean    mean       mean  
scenario               method                                    
Strong instrument      Linear 2SLS     0.771  16.669      0.016  
                       RF Calibrated   0.214  16.669      0.016  
                       RF Raw          0.425  16.669      0.016  
Very strong instrument Linear 2SLS     0.383  61.527      0.058  
                       RF Calibrated   0.187  61.527      0.058  
                       RF Raw          0.307  61.527      0.058  
Weak instrument        Linear 2SLS     8.370   3.697      0.004  
                       RF Calibrated   0.226   3.697      0.004  
                       RF Raw          0.495   3.697      0.004